11. Realizar un análisis de stopwords de las reviews. Dada la frecuencia de los tokens de las reviews, mostrar los 30 tokens más frecuentes y listar del total de tokens cuales son stopwords utilizando nltk. (⭐⭐)

In [1]:
!pip install pyspark
!apt update

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=cd06a1170c66ad9808049c3a0a7eba71370816f4cdb8e527cb8427227d6cd051
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-sec

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [5]:
sqlContext = SQLContext(sc)
# Atencion, utilizo estos metodos para separar bien la forma en la que se lee del csv, ya que sino se lee de manera incorrecta
app_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('/content/drive/MyDrive/aa-OrgaDeDatos/googleplaystore.csv', header=True, inferSchema=True)
app_reviews = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('/content/drive/MyDrive/aa-OrgaDeDatos/googleplaystore_user_reviews.csv', header=True, inferSchema=True)
rdd_app_data = app_data.rdd
rdd_app_reviews = app_reviews.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Me encargue previamente de revisar los valores de translated_review, por lo que filtro aquellos que no tienen sentido

In [6]:
rdd_reviews_cached = rdd_app_reviews.filter(lambda row: row["Translated_Review"] is not None and row["Translated_Review"].lower() != 'nan').distinct().cache()

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Voy a extender el modelo utilizando en clase (con unas modificaciones) para encontrar los dos registros cuyos valores eran los mayores ordenados de mayor a menor, pero para obtener los 30 tokens mas utilizados.

In [8]:
def takeSecond(elem):
    return elem[1] *-1

def thirtyMax(tokens):
  tokens.sort(key=takeSecond)
  return tokens[:30]

In [9]:
vocabulary = rdd_reviews_cached.flatMap(lambda row: word_tokenize(row["Translated_Review"]))
tokens_frequency_rdd = vocabulary.map(lambda row: (row, 1)).reduceByKey(lambda x, y: x+y)

thirty_most_repeated_tokens = tokens_frequency_rdd.map(lambda x: [(x[0], x[1])]).reduce(lambda x, y: thirtyMax(x + y))
thirty_most_repeated_tokens

[('.', 48630),
 ('I', 28205),
 (',', 20734),
 ('!', 10804),
 ('game', 6152),
 ('It', 5447),
 ("'s", 4081),
 ('like', 3855),
 ('app', 3725),
 ('The', 3552),
 ('it', 3394),
 ('get', 3352),
 ('time', 3252),
 ('good', 3185),
 ('This', 3034),
 ('...', 2995),
 ("n't", 2863),
 ('?', 2751),
 ('love', 2536),
 ('great', 2377),
 ('would', 2224),
 ('really', 1999),
 ('even', 1901),
 ('phone', 1879),
 ("'m", 1807),
 ('work', 1780),
 ('..', 1702),
 ('update', 1685),
 (')', 1675),
 ('ca', 1672)]

A continuacion el set de stopwords que estan presentes en la lista de los 30 tokens mas frecuentes

In [10]:
stopWordSet = set(stopwords.words('english'))
stopWords = {token for token, valor in thirty_most_repeated_tokens if token in stopWordSet}
stopWords

{'it'}

# UTILIZADO PARA COMPARAR

In [11]:
tokens_frequency_rdd.takeOrdered(30, lambda x: -x[1])

[('.', 48630),
 ('I', 28205),
 (',', 20734),
 ('!', 10804),
 ('game', 6152),
 ('It', 5447),
 ("'s", 4081),
 ('like', 3855),
 ('app', 3725),
 ('The', 3552),
 ('it', 3394),
 ('get', 3352),
 ('time', 3252),
 ('good', 3185),
 ('This', 3034),
 ('...', 2995),
 ("n't", 2863),
 ('?', 2751),
 ('love', 2536),
 ('great', 2377),
 ('would', 2224),
 ('really', 1999),
 ('even', 1901),
 ('phone', 1879),
 ("'m", 1807),
 ('work', 1780),
 ('..', 1702),
 ('update', 1685),
 (')', 1675),
 ('ca', 1672)]